In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

url = 'https://www.basketball-reference.com/leagues/NBA_2012_standings.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

east_standings = soup.find('table', {'id': 'confs_standings_E'})
if east_standings is None:
    east_standings = soup.find('table', {'id': 'divs_standings_E'})
    east_rows = east_standings.find_all('tr')[2:]
else:
    east_rows = east_standings.find_all('tr')[1:]
west_standings = soup.find('table', {'id': 'confs_standings_W'})
if west_standings is None:
    west_standings = soup.find('table', {'id': 'divs_standings_W'})
    west_rows = west_standings.find_all('tr')[2:]
else:
    west_rows = west_standings.find_all('tr')[1:]

data = []

for i in range(len(east_rows)):
    cols = east_rows[i].findAll('td')
    team = east_rows[i].find('a')
    if team is not None and len(cols) > 1:
        data.append([team['href'][7:10], cols[2].text])

    cols = west_rows[i].findAll('td')
    team = west_rows[i].find('a')
    if team is not None and len(cols) > 1:
        data.append([team['href'][7:10], cols[2].text])

data.sort(key=lambda x: float(x[1]), reverse=True)
for i in range(30):
    data[i] = [i + 1, data[i][0], data[i][1]]

df = pd.DataFrame(data, columns=['Rk', 'Tm', 'Overall'])
csv_content = df.to_csv(index=False)

with open('../data/2011-12 Standings.csv', mode='w') as file:
    file.write(csv_content)